In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Oct 19 09:19:21 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   27C    P0    44W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [4]:
import torch

!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.1+cu113.html
     |████████████████████████████████| 7.9 MB 4.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.1+cu113.html
     |████████████████████████████████| 3.5 MB 4.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.1+cu113.html
     |████████████████████████████████| 2.4 MB 4.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-i2uvxpko
  Running command git clone -q https://github.com/pyg-team/pytorch_geometric.git /tmp/pip-req-build-i2uvxpko
  Created wheel for torch-geomet

In [5]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 4.6 MB/s 
     |████████████████████████████████| 163 kB 93.3 MB/s 
     |████████████████████████████████| 7.6 MB 77.7 MB/s 


In [73]:
import pickle
import torch
import transformers
from transformers import BertTokenizer, BertModel

from transformers import AutoTokenizer, AutoModel, AutoConfig


from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.data import InMemoryDataset, download_url
from torch_geometric.data import Data

from torch_geometric.loader import DataLoader

import torch.nn.functional as F
import numpy as np

import math
import pandas as pd
import pip
import re

import os.path as osp

import torch
from torch_geometric.data import Dataset, download_url

import random

random.seed(123)

torch.manual_seed(123)
from sklearn.metrics import jaccard_score, f1_score, accuracy_score, recall_score, precision_score, confusion_matrix

np.random.seed(123)
torch.cuda.manual_seed_all(123)

In [74]:
with open("/content/drive/MyDrive/Colab Notebooks/27/self_report/two_disease/final_combined_all_self_report.p", "rb") as f:
  final_combined_all_dialog = pickle.load(f)

In [75]:
class MyOwnDataset_complete(InMemoryDataset):
    def __init__(self, root, transform = None, pre_transform = None, pre_filter = None):
        super().__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[0])
        
    @property
    def processed_file_names(self):
        return ['data_complete_1367.pt']

In [76]:
dataset_joint_kg = MyOwnDataset_complete(root = '/content/drive/MyDrive/Colab Notebooks/27/self_report/two_disease/joint_graph')
dataset_joint_kg

MyOwnDataset_complete(1367)

In [77]:
data_joint_kg = []
for i in dataset_joint_kg:
    data_joint_kg.append(i)

In [78]:
data_joint_kg

[Data(x=[4, 768], edge_index=[2, 3], edge_attr=[3], y=[1]),
 Data(x=[4, 768], edge_index=[2, 3], edge_attr=[3], y=[1]),
 Data(x=[4, 768], edge_index=[2, 3], edge_attr=[3], y=[1]),
 Data(x=[4, 768], edge_index=[2, 3], edge_attr=[3], y=[1]),
 Data(x=[4, 768], edge_index=[2, 3], edge_attr=[3], y=[1]),
 Data(x=[4, 768], edge_index=[2, 3], edge_attr=[3], y=[1]),
 Data(x=[4, 768], edge_index=[2, 3], edge_attr=[3], y=[1]),
 Data(x=[4, 768], edge_index=[2, 3], edge_attr=[3], y=[1]),
 Data(x=[3, 768], edge_index=[2, 2], edge_attr=[2], y=[1]),
 Data(x=[4, 768], edge_index=[2, 3], edge_attr=[3], y=[1]),
 Data(x=[4, 768], edge_index=[2, 3], edge_attr=[3], y=[1]),
 Data(x=[4, 768], edge_index=[2, 3], edge_attr=[3], y=[1]),
 Data(x=[4, 768], edge_index=[2, 3], edge_attr=[3], y=[1]),
 Data(x=[4, 768], edge_index=[2, 3], edge_attr=[3], y=[1]),
 Data(x=[4, 768], edge_index=[2, 3], edge_attr=[3], y=[1]),
 Data(x=[4, 768], edge_index=[2, 3], edge_attr=[3], y=[1]),
 Data(x=[4, 768], edge_index=[2, 3], edg

In [79]:
len(data_joint_kg)

1367

In [80]:
tokenizer  = AutoTokenizer.from_pretrained("cambridgeltl/SapBERT-from-PubMedBERT-fulltext")
bert_model = AutoModel.from_pretrained("cambridgeltl/SapBERT-from-PubMedBERT-fulltext")
bert_model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [81]:
print(tokenizer.all_special_tokens)
print(tokenizer.all_special_ids)
p = {
        'additional_special_tokens' : ['[DOC]', '[PAT]', '[SR_START]', '[SR_END]']
    }

tokenizer.add_special_tokens(p)
print(tokenizer.convert_tokens_to_ids('[DOC]'))
print(tokenizer.convert_tokens_to_ids('[PAT]'))
print(tokenizer.convert_tokens_to_ids('[SR_START]'))
print(tokenizer.convert_tokens_to_ids('[SR_END]'))
print(len(tokenizer))
print(bert_model.resize_token_embeddings(len(tokenizer)))
print(tokenizer.all_special_tokens)
print(tokenizer.all_special_ids)

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']
[1, 3, 0, 2, 4]
30522
30523
30524
30525
30526
Embedding(30526, 768)
['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]', '[DOC]', '[PAT]', '[SR_START]', '[SR_END]']
[1, 3, 0, 2, 4, 30522, 30523, 30524, 30525]


In [82]:
len(data_joint_kg)

1367

In [83]:
final_combined_all_dialog[0]

{'first_utterance': "[SR_START] [PAT] : Doctor, I see Spots or clouds in my vision. I can't see anything clearly. I feel helpless. [SR_END]",
 'label': 'central retinal artery or vein occlusion'}

In [84]:
data_combined = []

for i in range(len(data_joint_kg)):
    temp_list = []
    temp_list.append(data_joint_kg[i])
#     temp_list.append(input_ids[i,:])
#     temp_list.append(attention_mask[i,:])
    temp_list.append(final_combined_all_dialog[i]['first_utterance'])
    data_combined.append(temp_list)

In [85]:
len(data_combined)

1367

In [86]:
len(data_combined[0])

2

In [87]:
data_combined[0]

[Data(x=[4, 768], edge_index=[2, 3], edge_attr=[3], y=[1]),
 "[SR_START] [PAT] : Doctor, I see Spots or clouds in my vision. I can't see anything clearly. I feel helpless. [SR_END]"]

In [88]:
data_combined[0][1]

"[SR_START] [PAT] : Doctor, I see Spots or clouds in my vision. I can't see anything clearly. I feel helpless. [SR_END]"

In [89]:
random.shuffle(data_combined)

In [90]:
joint_data_after_shuffling = []
for i in data_combined:
    joint_graph = i[0]
#     print(joint_graph)
#     print("\n")
    joint_data_after_shuffling.append(joint_graph)

In [91]:
len(joint_data_after_shuffling)

1367

In [92]:
joint_data_after_shuffling[0]

Data(x=[4, 768], edge_index=[2, 3], edge_attr=[3], y=[1])

In [93]:
joint_data_after_shuffling[0].y

tensor([53.])

In [94]:
joint_data_after_shuffling_train = joint_data_after_shuffling[:957]
len(joint_data_after_shuffling_train)

957

In [95]:
joint_data_after_shuffling_valid = joint_data_after_shuffling[957:1094]
len(joint_data_after_shuffling_valid)

137

In [96]:
joint_data_after_shuffling_test = joint_data_after_shuffling[1094:]
len(joint_data_after_shuffling_test)

273

In [97]:
dialog_after_shuffling = []
for i in data_combined:
    dlog = i[1]
    print(dlog)
    dialog_after_shuffling.append(dlog)
    print()
    

[SR_START] [PAT] : Hello! Doctor. I have got Skin lesion. I am a little worried about it. [SR_END]

[SR_START] [PAT] : Doctor, I have got Fatigue. I don't feel energetic in any condition, help me. [SR_END]

[SR_START] [PAT] : Hello doctor, I am suffering from Fatigue. [SR_END]

[SR_START] [PAT] : Hi Doctor, why am I having Fever. I have to get well before next week. I have my final exams. [SR_END]

[SR_START] [PAT] : Hello doctor, I am struggling with Shortness of breath. I donot want to make this severe. Please help. [SR_END]

[SR_START] [PAT] : Hi doctor. I have got Leg pain. Would you please tell me its cause? I can't walk. I am unable to move even to the outside of our house. [SR_END]

[SR_START] [PAT] : I am ill with Dizziness. I cannot concentrate while studying. [SR_END]

[SR_START] [PAT] : Doctor, I am ill with Weakness. I need to gain my strength before my exam. [SR_END]

[SR_START] [PAT] : Doctor, I am dealing with Abnormal involuntary movements. [SR_END]

[SR_START] [PAT] : 

In [98]:
len(dialog_after_shuffling)

1367

In [99]:
dialog_after_shuffling[0]

'[SR_START] [PAT] : Hello! Doctor. I have got Skin lesion. I am a little worried about it. [SR_END]'

In [100]:
dialog_after_shuffling_train = dialog_after_shuffling[:957]
len(dialog_after_shuffling_train)

957

In [101]:
dialog_after_shuffling_valid = dialog_after_shuffling[957:1094]
len(dialog_after_shuffling_valid)

137

In [102]:
dialog_after_shuffling_test = dialog_after_shuffling[1094:]
len(dialog_after_shuffling_test)

273

In [103]:
enc_train = tokenizer(dialog_after_shuffling_train, padding='max_length', max_length=512, truncation=True, return_tensors = 'pt')
enc_train

{'input_ids': tensor([[    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        ...,
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [104]:
enc_valid = tokenizer(dialog_after_shuffling_valid, padding='max_length', max_length=512, truncation=True, return_tensors= 'pt')
enc_valid

{'input_ids': tensor([[    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        ...,
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [105]:
enc_test = tokenizer(dialog_after_shuffling_test, padding='max_length', max_length=512, truncation=True, return_tensors='pt')
enc_test

{'input_ids': tensor([[    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        ...,
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [106]:
# x1 - Joint Graph data
# x2 - input ids
# x3 - attention mask
class Dataset_Knowledge_Linear(Dataset):
    def __init__(self, x1, x2, x3):
        self.x1 = x1
        self.x2 = x2
        self.x3 = x3
    
    def __len__(self):
        return len(self.x1)
    
    def __getitem__(self, idx):
        x1_idx = self.x1[idx]
        x2_idx = self.x2[idx]
        x3_idx = self.x3[idx]
        
        return x1_idx, x2_idx, x3_idx
        

In [107]:
train_loader = DataLoader(Dataset_Knowledge_Linear(joint_data_after_shuffling_train, enc_train['input_ids'], enc_train['attention_mask']), batch_size = 16, shuffle = True)
train_loader




In [108]:
valid_loader = DataLoader(Dataset_Knowledge_Linear(joint_data_after_shuffling_valid, enc_valid['input_ids'], enc_valid['attention_mask']), batch_size = 16, shuffle=False)
valid_loader



In [109]:
test_loader = DataLoader(Dataset_Knowledge_Linear(joint_data_after_shuffling_test, enc_test['input_ids'], enc_test['attention_mask']), batch_size=16, shuffle=False)
test_loader



In [110]:
for step,data in enumerate(train_loader):
    print("Step : ", step)
    print("Data : ", data)
    print()

Step :  0
Data :  [DataBatch(x=[64, 768], edge_index=[2, 48], edge_attr=[48], y=[16], batch=[64], ptr=[17]), tensor([[    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        ...,
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0]]), tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])]

Step :  1
Data :  [DataBatch(x=[63, 768], edge_index=[2, 47], edge_attr=[47], y=[16], batch=[63], ptr=[17]), tensor([[    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        ...,
        [   

In [111]:
for step,data in enumerate(valid_loader):
    print("Step : ", step)
    print("Data : ", data)
    print("Label : ", data[0].y)
    print()

Step :  0
Data :  [DataBatch(x=[66, 768], edge_index=[2, 52], edge_attr=[52], y=[16], batch=[66], ptr=[17]), tensor([[    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        ...,
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0]]), tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])]
Label :  tensor([12., 26., 71., 18., 20., 19., 42., 33., 45., 76., 44., 61.,  5., 68.,
        70., 61.])

Step :  1
Data :  [DataBatch(x=[64, 768], edge_index=[2, 48], edge_attr=[48], y=[16], batch=[64], ptr=[17]), tensor([[    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,

In [112]:
for step,data in enumerate(test_loader):
    print("Step : ", step)
    print("Data : ", data)
    print()

Step :  0
Data :  [DataBatch(x=[64, 768], edge_index=[2, 48], edge_attr=[48], y=[16], batch=[64], ptr=[17]), tensor([[    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        ...,
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0]]), tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])]

Step :  1
Data :  [DataBatch(x=[63, 768], edge_index=[2, 47], edge_attr=[47], y=[16], batch=[63], ptr=[17]), tensor([[    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        ...,
        [   

In [113]:
class CustomBert(torch.nn.Module):
    def __init__(self):
        super(CustomBert, self).__init__()
        self.bert = bert_model
    
    def forward(self, ids, mask):
        output_encoding = self.bert(ids, mask)
        
        cls = output_encoding['last_hidden_state'].permute(1,0,2)[0]
        
        return cls

In [114]:
class GAT_joint(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, args):
        super(GAT_joint, self).__init__()
        
        self.conv1 = GATConv(input_dim, hidden_dim, heads = args['heads'])
        self.conv2 = GATConv(args['heads']*hidden_dim, hidden_dim, heads = args['heads'])
        
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        
        x = self.conv1(x, edge_index)
        x = F.dropout(F.relu(x), p=args['dropout'], training = self.training)
        
        x = self.conv2(x, edge_index)
        x = F.dropout(F.relu(x), p=args['dropout'], training = self.training)
        
        x = global_mean_pool(x, batch)
        x = F.dropout(F.relu(x), p=args['dropout'], training = self.training)
        
        return x
        

In [115]:
class Linear_Knowledge(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, args):
        super(Linear_Knowledge, self).__init__()
        
        self.custom_bert = CustomBert()
        self.gat_joint = GAT_joint(input_dim, hidden_dim, args)
        # self.linear1 = torch.nn.Linear(1152, 384)
        # self.linear2 = torch.nn.Linear(384, output_dim)
        
        self.temp_linear = torch.nn.Linear(1152, output_dim)

    def forward(self, data, input_ids, attention_mask):
        bert_output = self.custom_bert(input_ids, attention_mask)
        gat_output = self.gat_joint(data)
        
        final = torch.cat((bert_output, gat_output), dim=1)
        
        # final = self.linear1(final)
        # final = F.relu(final)
        
        # final = self.linear2(final)

        final = self.temp_linear(final)
        
        return final

In [116]:
args = {
    'heads' : 3,
    'dropout' : 0.5
}

In [117]:
model = Linear_Knowledge(input_dim = 768, hidden_dim = 128, output_dim = 90, args = args)
model

Linear_Knowledge(
  (custom_bert): CustomBert(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30526, 768)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (Layer

In [118]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [119]:
model = model.to(device)
model

Linear_Knowledge(
  (custom_bert): CustomBert(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30526, 768)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (Layer

In [120]:
model.parameters()

<generator object Module.parameters at 0x7f2d4af1d750>

In [121]:
total_parameters = sum(p.numel() for p in model.parameters())
total_parameters/1e6

110.033754

In [122]:
total_train_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_train_parameters/1e6

110.033754

In [123]:
for i,name in enumerate(model.named_parameters()):
    print(i," : ",name[0])

0  :  custom_bert.bert.embeddings.word_embeddings.weight
1  :  custom_bert.bert.embeddings.position_embeddings.weight
2  :  custom_bert.bert.embeddings.token_type_embeddings.weight
3  :  custom_bert.bert.embeddings.LayerNorm.weight
4  :  custom_bert.bert.embeddings.LayerNorm.bias
5  :  custom_bert.bert.encoder.layer.0.attention.self.query.weight
6  :  custom_bert.bert.encoder.layer.0.attention.self.query.bias
7  :  custom_bert.bert.encoder.layer.0.attention.self.key.weight
8  :  custom_bert.bert.encoder.layer.0.attention.self.key.bias
9  :  custom_bert.bert.encoder.layer.0.attention.self.value.weight
10  :  custom_bert.bert.encoder.layer.0.attention.self.value.bias
11  :  custom_bert.bert.encoder.layer.0.attention.output.dense.weight
12  :  custom_bert.bert.encoder.layer.0.attention.output.dense.bias
13  :  custom_bert.bert.encoder.layer.0.attention.output.LayerNorm.weight
14  :  custom_bert.bert.encoder.layer.0.attention.output.LayerNorm.bias
15  :  custom_bert.bert.encoder.layer.0.in

In [124]:
for p in model.parameters():
    p.requires_grad = False

In [125]:
total_train_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_train_parameters/1e6

0.0

In [126]:
cnt=0
for name, param in model.named_parameters():
    if(param.requires_grad == False and cnt>=199):
        param.requires_grad = True
        print(name)
#         print()
    cnt+=1    

gat_joint.conv1.att_src
gat_joint.conv1.att_dst
gat_joint.conv1.bias
gat_joint.conv1.lin_src.weight
gat_joint.conv2.att_src
gat_joint.conv2.att_dst
gat_joint.conv2.bias
gat_joint.conv2.lin_src.weight
temp_linear.weight
temp_linear.bias


In [127]:
total_train_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_train_parameters/1e6

0.548442

In [128]:
1e-3

0.001

In [129]:
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
# optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)
criterion = torch.nn.CrossEntropyLoss()


In [130]:
def train(loader):
    model.train()
    
    for data in loader:
        optimizer.zero_grad()
        
        graph_data = data[0]
        input_ids = data[1]
        attention_mask = data[2]
        
        graph_data = graph_data.to(device)
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        
        out = model(graph_data, input_ids, attention_mask)
        
        z = graph_data.y
        ground_truth = z.long()
        
        loss = criterion(out, ground_truth)
        
        loss.backward()
        optimizer.step()

In [131]:
def test(loader):
    model.eval()

    y_true = []
    y_pred = []
    
    correct = 0
    for data in loader:
        with torch.no_grad():
            
            graph_data = data[0]
            input_ids = data[1]
            attention_mask = data[2]

            graph_data = graph_data.to(device)
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            
            out = model(graph_data, input_ids, attention_mask)
            
            pred = out.argmax(dim=1)
            
            correct += int((pred == graph_data.y).sum())

            true_label = graph_data.y

            y_true.extend(true_label.tolist())
            y_pred.extend(pred.tolist())
            
    return correct/len(loader.dataset) , y_true, y_pred       

In [132]:
q1=torch.tensor([1,2,3,4,5])
q1

tensor([1, 2, 3, 4, 5])

In [133]:
q2=torch.tensor([2,2,2,4,6])
q2

tensor([2, 2, 2, 4, 6])

In [134]:
int((q1 == q2).sum())

2

In [ ]:
for epoch in range(25):
    train(train_loader)
    train_acc, y_true_train, y_pred_train = test(train_loader)
    valid_acc, y_true_valid, y_pred_valid = test(valid_loader)
    print(f'Epoch : {epoch : 03d}, Train acc : {train_acc : .4f}, Valid acc : {valid_acc : .4f}')

Epoch :  00, Train acc :  0.2644, Valid acc :  0.1752
Epoch :  01, Train acc :  0.4389, Valid acc :  0.2701
Epoch :  02, Train acc :  0.5110, Valid acc :  0.2482
Epoch :  03, Train acc :  0.5350, Valid acc :  0.2993
Epoch :  04, Train acc :  0.5956, Valid acc :  0.2774
Epoch :  05, Train acc :  0.6499, Valid acc :  0.3285
Epoch :  06, Train acc :  0.6823, Valid acc :  0.2920
Epoch :  07, Train acc :  0.7220, Valid acc :  0.2774
Epoch :  08, Train acc :  0.7450, Valid acc :  0.2847
Epoch :  09, Train acc :  0.7785, Valid acc :  0.2555
Epoch :  10, Train acc :  0.7764, Valid acc :  0.2409
Epoch :  11, Train acc :  0.8391, Valid acc :  0.2409
Epoch :  12, Train acc :  0.8548, Valid acc :  0.2263
Epoch :  13, Train acc :  0.8715, Valid acc :  0.2409
Epoch :  14, Train acc :  0.8840, Valid acc :  0.2555
Epoch :  15, Train acc :  0.8966, Valid acc :  0.2336
Epoch :  16, Train acc :  0.9007, Valid acc :  0.2482
Epoch :  17, Train acc :  0.9143, Valid acc :  0.2847
Epoch :  18, Train acc :  0.

In [ ]:
test_acc, y_true_test, y_pred_test = test(test_loader)
print("Test acc : ", test_acc)

In [ ]:
print("=========Using Sklearn======")
print("\nConfusion matrix\n", confusion_matrix(y_true_test, y_pred_test))
print("\nAccuracy\n", accuracy_score(y_true_test, y_pred_test))
print("\nF1 score\n", f1_score(y_true_test, y_pred_test, average = 'macro'))
print("\nJaccard similarity\n", jaccard_score(y_true_test, y_pred_test, average = 'macro'))

In [ ]:
z = 65.59 + 64.15 + 63.08
z

In [ ]:
z/3